In [2]:
import pandas as pd
import numpy as np

#### We are using our Spam/Ham Dataset. This time we are applying few different models and use VotingClassifier to classify outputs.

In [3]:
url = "https://raw.githubusercontent.com/ga-students/SF-DAT-20/master/Data/spambase.csv"
SpamData = pd.read_csv(url)
SpamData.head()
len(SpamData)

4601

In [4]:
ListOfAllVariables = SpamData.columns.values
print ListOfAllVariables
X = SpamData[ListOfAllVariables]
del X['is_spam']
y = SpamData['is_spam']

['word_freq_make' 'word_freq_address' 'word_freq_all' 'word_freq_3d'
 'word_freq_our' 'word_freq_over' 'word_freq_remove' 'word_freq_internet'
 'word_freq_order' 'word_freq_mail' 'word_freq_receive' 'word_freq_will'
 'word_freq_people' 'word_freq_report' 'word_freq_addresses'
 'word_freq_free' 'word_freq_business' 'word_freq_email' 'word_freq_you'
 'word_freq_credit' 'word_freq_your' 'word_freq_font' 'word_freq_000'
 'word_freq_money' 'word_freq_hp' 'word_freq_hpl' 'word_freq_george'
 'word_freq_650' 'word_freq_lab' 'word_freq_labs' 'word_freq_telnet'
 'word_freq_857' 'word_freq_data' 'word_freq_415' 'word_freq_85'
 'word_freq_technology' 'word_freq_1999' 'word_freq_parts' 'word_freq_pm'
 'word_freq_direct' 'word_freq_cs' 'word_freq_meeting' 'word_freq_original'
 'word_freq_project' 'word_freq_re' 'word_freq_edu' 'word_freq_table'
 'word_freq_conference' 'char_freq_;' 'char_freq_(' 'char_freq_['
 'char_freq_!' 'char_freq_$' 'char_freq_#' 'capital_run_length_average'
 'capital_run_lengt

In [4]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier  #VotingClassifier is part of sklearn.ensemble

In [5]:
clf1 = LogisticRegression()
clf2 = RandomForestClassifier(max_depth=5, n_estimators = 1000)
clf3 = BernoulliNB()
clf4 = MultinomialNB()
clf5 = GaussianNB()

In [6]:
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('bnb', clf3),('mnb',clf4),('gnb',clf5)], voting='hard')

#### When voting is set to 'hard', then the outcome will simply becomes the majority vote. i.e., if 3 classifiers predict class 1 and 2 classifiers predict class 2, majority vote is class 1. 

In [7]:
for clf in [clf1,clf2,clf3,clf4,clf5,eclf]:
    scores = cross_validation.cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print scores.mean()


0.912391909726
0.918253684137
0.880428497671
0.781969260361
0.825647242022
0.924336394537


In [8]:
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('bnb', clf3),('mnb',clf4),('gnb',clf5)], voting='soft')
for clf in [clf1,clf2,clf3,clf4,clf5,eclf]:
    scores = cross_validation.cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print scores.mean()

0.912391909726
0.919776133951
0.880428497671
0.781969260361
0.825647242022
0.916071734489


#### When voting is set to 'soft', then the outcome will be picked based on predictions on probabilities. For instance, imagine classifier 1 and 2 predict 49% for class 1 and 51% for class 2. Also, assume classifier 3 predicts 90% probability for class 1 and 10% for class 2. In 'soft' voting method, the prediction is class 1. However, if we were using 'hard' method, then our prediction would be class 2.

# Using the VotingClassifier with GridSearch

In [10]:
from sklearn.grid_search import GridSearchCV
clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = BernoulliNB()
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('bnb', clf3)],voting='hard')
params = {'lr__C': [0.1,1,10],
          'rf__n_estimators':[1000],
          'rf__max_depth':[2,5,10],
          'bnb__alpha':[0.1,0.5,1]}
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
gridfit = grid.fit(X, y)

In [11]:
print gridfit.best_params_

{'bnb__alpha': 0.1, 'rf__max_depth': 10, 'rf__n_estimators': 1000, 'lr__C': 10}


In [12]:
print gridfit.grid_scores_
print gridfit.best_score_

[mean: 0.90502, std: 0.02864, params: {'bnb__alpha': 0.1, 'rf__max_depth': 2, 'rf__n_estimators': 1000, 'lr__C': 0.1}, mean: 0.91915, std: 0.03682, params: {'bnb__alpha': 0.1, 'rf__max_depth': 5, 'rf__n_estimators': 1000, 'lr__C': 0.1}, mean: 0.92023, std: 0.04006, params: {'bnb__alpha': 0.1, 'rf__max_depth': 10, 'rf__n_estimators': 1000, 'lr__C': 0.1}, mean: 0.90589, std: 0.03054, params: {'bnb__alpha': 0.1, 'rf__max_depth': 2, 'rf__n_estimators': 1000, 'lr__C': 1}, mean: 0.92110, std: 0.03885, params: {'bnb__alpha': 0.1, 'rf__max_depth': 5, 'rf__n_estimators': 1000, 'lr__C': 1}, mean: 0.92263, std: 0.04123, params: {'bnb__alpha': 0.1, 'rf__max_depth': 10, 'rf__n_estimators': 1000, 'lr__C': 1}, mean: 0.90676, std: 0.03107, params: {'bnb__alpha': 0.1, 'rf__max_depth': 2, 'rf__n_estimators': 1000, 'lr__C': 10}, mean: 0.92132, std: 0.03890, params: {'bnb__alpha': 0.1, 'rf__max_depth': 5, 'rf__n_estimators': 1000, 'lr__C': 10}, mean: 0.92436, std: 0.04105, params: {'bnb__alpha': 0.1, 'rf_